In [ ]:
import csv
import glob
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import re

from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, classification_report

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchmetrics import AUROC

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
pl.seed_everything(RANDOM_SEED)

# 1. MELD 

**Column Specification:**

**Column Name = Description** 

Sr No. = Serial numbers of the utterances mainly for referencing the utterances in case of different versions or multiple copies with different subsets.                

Utterance = Individual utterances from EmotionLines as a string.                                           

Speaker = Name of the speaker associated with the utterance.                                             

Emotion = The emotion (neutral, joy, sadness, anger, surprise, fear, disgust)in the utterance.

Sentiment = The sentiment (positive, neutral, negative) expressed by the speaker in the utterance.         

Dialogue_ID = The index of the dialogue starting from 0.                                                     

Utterance_ID = The index of the particular utterance in the dialogue starting from 0.                         

Season = The season no. of Friends TV Show to which a particular utterance belongs.

**Data Splits:**

Train = 9989

Dev = 1109

Test = 2610

Total = 13708

## Train

In [ ]:
#download the data here = https://affective-meld.github.io/
df_train = pd.read_csv(r"C:\Data\Alternate data\MELD\train_sent_emo.csv") #please put your path

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
y = pd.get_dummies(df_train.Sentiment)
print(y.head())

In [ ]:
df_train2 = df_train[['Utterance', 'Sentiment']]

In [ ]:
df_train3 = pd.concat([df_train2, y], axis=1)
df_train3.head(5)

In [ ]:
df_train3['Sentiment'] = df_train3['Sentiment'].astype('category')

encode_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

df_train3['Sentiment'].replace(encode_map, inplace=True)

In [ ]:
df_train_final = df_train3[['Utterance', 'Sentiment']]
df_train_final.head(5)

## Dev

In [ ]:
df_dev = pd.read_csv(r"C:\Data\Alternate data\MELD\dev_sent_emo.csv")

In [ ]:
df_dev.shape

In [ ]:
df_dev.head(10)

In [ ]:
w = pd.get_dummies(df_dev.Sentiment)
print(w.head())

In [ ]:
df_dev2 = df_dev[['Utterance', 'Sentiment']]

In [ ]:
df_dev3 = pd.concat([df_dev2, w], axis=1)
df_dev3.head()

In [ ]:
df_dev3['Sentiment'] = df_dev3['Sentiment'].astype('category')

encode_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

df_dev3['Sentiment'].replace(encode_map, inplace=True)

In [ ]:
df_dev_final = df_dev3[['Utterance', 'Sentiment']]
df_dev_final.head(5)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"gimme", "give me ", text)
    text = re.sub(r"'em'", "them ", text)
    text = re.sub(r"Y'", "You ", text)
    text = re.sub(r"'s", " ", text)
    text = re.sub(r"'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"re", " are ", text)
    text = re.sub(r"'d", " would ", text)
    text = re.sub(r"'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

## Test 

In [ ]:
df_test = pd.read_csv(r"C:\Data\Alternate data\MELD\test_sent_emo.csv")

In [ ]:
df_test.shape

In [ ]:
df_test.head(10)

In [ ]:
x = pd.get_dummies(df_test.Sentiment)
print(x.head())

In [ ]:
df_test2 = df_test[['Utterance', 'Sentiment']]

In [ ]:
df_test3 = pd.concat([df_test2, x], axis=1)
df_test3.head(5)

In [ ]:
df_test3['Sentiment'] = df_test3['Sentiment'].astype('category')

encode_map = {
    'negative': 0,
    'neutral': 1,
    'positive': 2
}

df_test3['Sentiment'].replace(encode_map, inplace=True)


In [ ]:
df_test_final = df_test3[['Utterance', 'Sentiment']]
df_test_final.head(5)

# EDA

In [ ]:
fr = [df_train2, df_dev2, df_test2]
comb = pd.concat(fr)
plot = comb.reset_index(drop=True)

In [ ]:
sns.countplot(x = 'Sentiment', data=plot)

# Preprocessing

In [ ]:
frames = [df_train_final, df_dev_final, df_test_final]
combine = pd.concat(frames)
all_data = combine.reset_index(drop=True)

In [ ]:
len(all_data)

In [ ]:
df_all = all_data[['Utterance', 'Sentiment']] #neg=0, neu=1, pos=2
df_all.tail(5) 

In [ ]:
LABEL_COLUMNS = df_all.columns.tolist()[1:]

In [ ]:
LABEL_COLUMNS

In [ ]:
df_all['Sentiment'].value_counts() #neg=0, neu=1, pos=2

# Tokenization

In [ ]:
MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# Number of tokens

In [ ]:
token_counts = []

for _, row in all_data.iterrows():
    token_count = len(tokenizer.encode(
        row["Utterance"],
        max_length=512,
        truncation=True
    ))

    token_counts.append(token_count)

In [ ]:
sns.histplot(token_counts)
plt.xlim([0, 60]);

**Most of the utterances contain less than 60 tokens. Therefore, the limit set will be 60.**

In [ ]:
MAX_TOKEN_COUNT = 60

## Tokenization process for the whole dataset

In [ ]:
class Dataset(Dataset):
    
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: BertTokenizer,
        max_token_len: int = 60
    ):

        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        
        data_row = self.data.iloc[index]
        
        Utterance = data_row.Utterance
        labels = data_row[LABEL_COLUMNS]  ##all

        encoding = self.tokenizer.encode_plus(
            Utterance,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return dict(
            Utterance=Utterance,
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            labels=torch.LongTensor(labels)

        )
    

## Sample:

In [ ]:
#Sample item from the dataset
train_dataset = Dataset(
  df_train_final,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT

)

sample_item = train_dataset[0]
sample_item.keys()

In [ ]:
sample_item["Utterance"]

In [ ]:
sample_item["labels"]

In [ ]:
sample_item["input_ids"].shape

In [ ]:
sample_item["attention_mask"].squeeze()[:27]

In [ ]:
sample_item

## Loading BERT model into a sample_item data

In [ ]:
bert_model = BertModel.from_pretrained(MODEL_NAME, return_dict=True)

sample_batch = next(iter(DataLoader(train_dataset, batch_size=16)))

sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape, sample_batch["labels"].shape

In [ ]:
output = bert_model(sample_batch["input_ids"], sample_batch["attention_mask"])
output.last_hidden_state.shape, output.pooler_output.shape

In [ ]:
bert_model.config.hidden_size 

**768 dimension comes from the BERT hidden size. 
The larger verison of BERT has more attentions heads and larger hidden size.**

# Encapsulating all data

In [ ]:
class DataModule(pl.LightningDataModule):
    
    def __init__(self, df_train_final, df_test_final, df_dev_final, tokenizer, batch_size=16, max_token_len=60):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = df_train_final
        self.test_df = df_test_final
        self.dev_df = df_dev_final
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = Dataset(
          self.train_df,
          self.tokenizer,
          self.max_token_len
        )

        self.dev_dataset = Dataset(
          self.dev_df,
          self.tokenizer,
          self.max_token_len
        )
        
        self.test_dataset = Dataset(
          self.test_df,
          self.tokenizer,
          self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
          self.train_dataset,
          batch_size=self.batch_size,
          shuffle=True,
          num_workers=0
        )

    def val_dataloader(self):
        return DataLoader(
          self.dev_dataset,
          batch_size=self.batch_size,
          num_workers=0
        )

    def test_dataloader(self):
        return DataLoader(
          self.test_dataset,
          batch_size=self.batch_size,
          num_workers=0
        )

**Instance for the data module:**

In [ ]:
N_EPOCHS = 1
BATCH_SIZE = 16

data_module = DataModule(df_train_final, df_dev_final, df_test_final, 
                         tokenizer, batch_size=BATCH_SIZE,max_token_len=MAX_TOKEN_COUNT)

# Modelling

**This model is a pre-trained BertModel and a linear layer to convert the BERT representation to a classification task.**

In [ ]:
class Tagger(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.bert = BertModel.from_pretrained(MODEL_NAME, return_dict=True)
        for parameter in self.bert.parameters():
            parameter.require_grad = False
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.CrossEntropyLoss()
        

    def forward(self, input_ids, attention_mask, labels=None):

        output = self.bert(input_ids, attention_mask=attention_mask)
        output = self.classifier(output.pooler_output)
        output = torch.softmax(output, dim=1)
        loss = 0
        
        #Print to debug
        #print(output)
        #print(labels)
        
        if labels is not None:
            labels = labels.flatten() ##
            loss = self.criterion(output, labels)
        return loss, output
    

    def training_step(self, batch, batch_idx):

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def validation_step(self, batch, batch_idx):

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def training_epoch_end(self, outputs):

        labels = []
        predictions = []

        for output in outputs:
            for out_labels in output["labels"].detach().cpu():
                labels.append(out_labels)
            for out_predictions in output["predictions"].detach().cpu():
                predictions.append(out_predictions)
        
        labels = torch.stack(labels).int()
        predictions = torch.stack(predictions)
        pred = torch.argmax(predictions, dim=1)
        
        train_acc = accuracy(pred, labels, num_classes=3)
        #print("Label:", labels)
        #print("Prediction:", pred)
        print("Training Accuracy:", train_acc)
        
        label = labels.flatten()
        auroc = AUROC(num_classes=3)
        auroc = auroc(predictions, label)
        print("AUROC:", auroc)
        

    def configure_optimizers(self): #configuring the optimizers

        optimizer = AdamW(self.parameters(), lr=2e-5)

        scheduler = get_linear_schedule_with_warmup(
          optimizer,
          num_warmup_steps=self.n_warmup_steps,
          num_training_steps=self.n_training_steps

        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )

        )

# Training

In [ ]:
steps_per_epoch=len(df_train_final) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [ ]:
#1/5 of the training steps as warm-up
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

**Instance of the model**

In [ ]:
model = Tagger(
  n_classes=3,
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps
)

Checkpointing that saves the best model (based on validation loss):

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="C:/Users/id301281/NLP/NLU/MELD",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
    )

And early stopping triggers when the loss hasn’t improved for the last 2 epochs:

In [ ]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

Starting training process:

In [ ]:
trainer = pl.Trainer(
    #logger=logger,
    #checkpoint_callback=checkpoint_callback,
    #callbacks=[early_stopping_callback],
    callbacks=[checkpoint_callback, early_stopping_callback],
    #checkpoint_callback = True,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)

In [ ]:
torch.cuda.is_available()

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trainer.test()

# Prediction

In [ ]:
trained_model = Tagger.load_from_checkpoint(
  trainer.checkpoint_callback.best_model_path,
  n_classes=3
)

trained_model.eval()
trained_model.freeze()

### Test 1 

In [ ]:
test_comment = "Hi, I'm Tracy, How can I help?"

encoding = tokenizer.encode_plus(

  test_comment,
  add_special_tokens=True,
  max_length=60,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)

_, test_prediction = trained_model(encoding["input_ids"], encoding["attention_mask"])
test_prediction = test_prediction.flatten().numpy()


for i, prediction in zip(encode_map, test_prediction):
    print(f"{i}: {prediction}")

### Test 2

In [ ]:
THRESHOLD = 0.5

test_comment = "You are such a loser! You'll regret everything you've done to me!"

encoding = tokenizer.encode_plus(
    test_comment,
    add_special_tokens=True,
    max_length=512,
    return_token_type_ids=False,
    padding="max_length",
    return_attention_mask=True,
    return_tensors='pt',
)

_, test_prediction = trained_model(encoding["input_ids"], encoding["attention_mask"])
test_prediction = test_prediction.flatten().numpy()

for i, prediction in zip(encode_map, test_prediction):
    if prediction < THRESHOLD:
        continue
print(f"{i}: {prediction}")

# Evaluation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = trained_model.to(device)

val_dataset = Dataset(
  df_dev_final,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

preds = []
labels = []

for item in tqdm(val_dataset):
    _, prediction = trained_model(
        item["input_ids"].unsqueeze(dim=0).to(device),
        item["attention_mask"].unsqueeze(dim=0).to(device)
    )

    preds.append(prediction.flatten())
    labels.append(item["labels"].int())

preds = torch.stack(preds).detach().cpu()
labels = torch.stack(labels).detach().cpu()

### Accuracy

In [ ]:
accuracy(preds, labels)

### ROC

In [ ]:
print("AUROC per tag")

#for i, name in enumerate(encode_map):
label = labels.flatten()
auroc = AUROC(num_classes=3)
auroc = auroc(preds, label)
print(auroc)

### Classification report

In [ ]:
#y_pred = preds.numpy()
y_pred = torch.argmax(preds, dim=1)
y_test = labels.numpy()

In [ ]:
print(classification_report(y_test, y_pred, target_names=encode_map))

### Confusion matrix

In [ ]:
def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=encode_map, columns=encode_map)
show_confusion_matrix(df_cm)